### Import Packages 


In [ ]:
import pandas as pd
from cassandra.cluster import Cluster 

## session connection to cassandra cluster

In [ ]:
clstr = Cluster()
session = clstr.connect()

### session connection to establish communication with cassandra 

In [ ]:
session.execute("DROP KEYSPACE IF EXISTS bd23")

### printing all the keyspaces

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

### creating a keyspace

In [ ]:
session.execute("CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

### creating a table in keyspace bd23

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventory ( 
    SKU TEXT, 
    p_name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(warehouse_num, SKU)
);
""")

### Load Inventory Data

In [ ]:
df = pd.read_csv('inventory.csv')
df

In [ ]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, name ={row.p_name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

### Inserting data into the table 

In [ ]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd23.inventory (SKU, p_name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO bd23.inventory (SKU, p_name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


### Search for all products located at warehouse_num = 123 

In [ ]:
rows = session.execute("select (SKU, p_name, description) from bd23.inventory where warehouse_num = 123 ")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")


